In [21]:
import pandas as pd
import numpy as np


import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation

file = "C:/Users/TYS/Documents/study/Master/INSY662/group project/Real Code/INSY662-Group-Project/Data/dataset_demand.csv"
data = pd.read_csv(file)
data

,date,start_station_cluster,rideable_type,trip_count
0,2022-01-01,0,classic_bike,94
1,2022-01-01,0,electric_bike,196
2,2022-01-01,1,classic_bike,11
3,2022-01-01,1,electric_bike,8
4,2022-01-01,2,classic_bike,37
...,...,...,...,...
12299,2022-12-31,17,electric_bike,189
12300,2022-12-31,18,classic_bike,13
12301,2022-12-31,18,electric_bike,46
12302,2022-12-31,19,classic_bike,9


In [22]:
# Filter data for start_station_cluster 0
#cluster_data = data[data['start_station_cluster'] == 0]

# Separate data for classic bikes and electric bikes
# use data instead of cluster_data
classic_bike_data = data[data['rideable_type'] == 'classic_bike']
electric_bike_data = data[data['rideable_type'] == 'electric_bike']
all_bike_data = data

classic_bike_data.head(), electric_bike_data.head()

(         date  start_station_cluster rideable_type  trip_count
 0  2022-01-01                      0  classic_bike          94
 2  2022-01-01                      1  classic_bike          11
 4  2022-01-01                      2  classic_bike          37
 5  2022-01-01                      3  classic_bike         118
 7  2022-01-01                      4  classic_bike          65,
          date  start_station_cluster  rideable_type  trip_count
 1  2022-01-01                      0  electric_bike         196
 3  2022-01-01                      1  electric_bike           8
 6  2022-01-01                      3  electric_bike         262
 8  2022-01-01                      4  electric_bike          97
 9  2022-01-01                      6  electric_bike           8)

In [23]:
# Convert 'date' to datetime and set as index
classic_bike_data['date'] = pd.to_datetime(classic_bike_data['date'])
electric_bike_data['date'] = pd.to_datetime(electric_bike_data['date'])
all_bike_data['date'] = pd.to_datetime(all_bike_data['date'])
classic_bike_data.set_index('date', inplace=True)
electric_bike_data.set_index('date', inplace=True)
all_bike_data.set_index('date', inplace=True)

# Display the first few rows of each dataset
classic_bike_data.head(), electric_bike_data.head() 

C:\Users\TYS\AppData\Local\Temp\ipykernel_52100\3828104725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classic_bike_data['date'] = pd.to_datetime(classic_bike_data['date'])
C:\Users\TYS\AppData\Local\Temp\ipykernel_52100\3828104725.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  electric_bike_data['date'] = pd.to_datetime(electric_bike_data['date'])


(            start_station_cluster rideable_type  trip_count
 date                                                       
 2022-01-01                      0  classic_bike          94
 2022-01-01                      1  classic_bike          11
 2022-01-01                      2  classic_bike          37
 2022-01-01                      3  classic_bike         118
 2022-01-01                      4  classic_bike          65,
             start_station_cluster  rideable_type  trip_count
 date                                                        
 2022-01-01                      0  electric_bike         196
 2022-01-01                      1  electric_bike           8
 2022-01-01                      3  electric_bike         262
 2022-01-01                      4  electric_bike          97
 2022-01-01                      6  electric_bike           8)

In [24]:
# Make sure the data is sorted by date
classic_bike_data.sort_index(inplace=True)
electric_bike_data.sort_index(inplace=True)
all_bike_data.sort_index(inplace=True)

# Determine the split point for the classic bike data
split_point_classic = len(classic_bike_data) // 2

# Split the classic bike data into train and test
classic_bike_train = classic_bike_data.iloc[:split_point_classic]
classic_bike_test = classic_bike_data.iloc[split_point_classic:]

# Determine the split point for the electric bike data
split_point_electric = len(electric_bike_data) // 2

# Split the electric bike data into train and test
electric_bike_train = electric_bike_data.iloc[:split_point_electric]
electric_bike_test = electric_bike_data.iloc[split_point_electric:]

split_point_all = len(all_bike_data) // 2

all_bike_train = all_bike_data.iloc[:split_point_all]
all_bike_test = all_bike_data.iloc[split_point_all:]

# You now have the train and test sets for both classic and electric bikes

# Display the first few rows of each dataset
classic_bike_train.head(), electric_bike_train.head(),all_bike_train.head()


(            start_station_cluster rideable_type  trip_count
 date                                                       
 2022-01-01                      0  classic_bike          94
 2022-01-01                      1  classic_bike          11
 2022-01-01                      2  classic_bike          37
 2022-01-01                      3  classic_bike         118
 2022-01-01                      4  classic_bike          65,
             start_station_cluster  rideable_type  trip_count
 date                                                        
 2022-01-01                      0  electric_bike         196
 2022-01-01                      1  electric_bike           8
 2022-01-01                      3  electric_bike         262
 2022-01-01                      4  electric_bike          97
 2022-01-01                      6  electric_bike           8,
             start_station_cluster  rideable_type  trip_count
 date                                                        
 2022-01-01  

In [ ]:
from pmdarima import auto_arima
import pandas as pd
from pandas import Timedelta, Timestamp
from statsmodels.tsa.arima.model import ARIMA

# Assuming 'data' is your DataFrame containing all clusters and bike types
clusters = data['start_station_cluster'].unique()
arima_params = {}

for cluster in clusters:
    for bike_type in ['classic_bike', 'electric_bike']:
        # Define the key for storing in the dictionary
        key = (cluster, bike_type)

        # Filter data for the current cluster and bike type
        cluster_bike_data = all_bike_train[(all_bike_train['start_station_cluster'] == cluster) &
                                 (all_bike_train['rideable_type'] == bike_type)]['trip_count'].dropna()

        # Convert the series to float
        cluster_bike_data = cluster_bike_data.astype(float)

        # Convert the index to weekly period
        cluster_bike_data.index = pd.DatetimeIndex(cluster_bike_data.index).to_period('D')

        try:
            # Find the best ARIMA parameters
            model = auto_arima(cluster_bike_data,
                               seasonal=True,
                               trace=True,
                               error_action='warn',
                               suppress_warnings=False,
                               stepwise=True,
                               max_p=3, max_q=3, max_d=2,
                               max_order=None,
                               enforce_stationarity=True,
                               enforce_invertibility=True)

            # Store parameters
            arima_params[key] = {
                'order': model.order,
                'seasonal_order': model.seasonal_order,
                'aic': model.aic()
            }

            # Concise output
            print(f"Cluster {cluster} ({bike_type.capitalize()}): ARIMA{model.order}, Seasonal Order={model.seasonal_order}, AIC={model.aic()}")

        except Exception as e:
            print(f"Failed to find ARIMA model for {bike_type} in cluster {cluster}: {e}")

In [33]:
c_temp_0 = classic_bike_train[classic_bike_train['start_station_cluster'] == 0]
c_temp_0

,start_station_cluster,rideable_type,trip_count
date,,,
2022-01-01,0,classic_bike,94
2022-01-02,0,classic_bike,111
2022-01-03,0,classic_bike,125
2022-01-04,0,classic_bike,139
2022-01-05,0,classic_bike,174
...,...,...,...
2022-06-28,0,classic_bike,481
2022-06-29,0,classic_bike,450
2022-06-30,0,classic_bike,434


In [39]:
classic_start = len(classic_bike_train)
classic_end = len(classic_bike_data) + len(classic_bike_test) - 1
electric_start = len(electric_bike_train)
electric_end = len(electric_bike_data) + len(electric_bike_test) - 1


In [42]:
def demand_forecasting(cluster_to_predict, bike_type_to_predict, date_to_predict):
    # Ensure the date is in the proper format
    date_to_predict = pd.to_datetime(date_to_predict)

    # Select the appropriate training dataset
    if bike_type_to_predict == "electric_bike":
        cluster_temp = electric_bike_train[electric_bike_train['start_station_cluster'] == cluster_to_predict]
    elif bike_type_to_predict == "classic_bike":
        cluster_temp = classic_bike_train[classic_bike_train['start_station_cluster'] == cluster_to_predict]
    else:
        raise ValueError("Bike type must be 'electric_bike' or 'classic_bike'.")

    # Ensure there is enough data to fit the model
    if len(cluster_temp) == 0:
        raise ValueError(f"No data available for cluster {cluster_to_predict} and bike type {bike_type_to_predict}.")

    # Fit the ARIMA model
    model_order = arima_params[(cluster_to_predict, bike_type_to_predict)]['order']
    model = ARIMA(cluster_temp['trip_count'], order=model_order)
    results = model.fit()

    # Forecast up to the date of interest
    # Assuming the index of the cluster_temp is a DatetimeIndex and is in ascending order
    forecast_end = (date_to_predict - cluster_temp.index[0]).days
    forecast = results.get_prediction(start=0, end=forecast_end)

    # Extract the forecast for the specific date
    try:
        predicted_count = forecast.predicted_mean.loc[date_to_predict]
    except KeyError:
        raise KeyError(f"No prediction available for {date_to_predict}. Model forecast may not cover this date.")

    return predicted_count


In [48]:
# Example parameters
cluster_to_predict = 1  # This is the cluster number you want to predict for
bike_type_to_predict = 'classic_bike'  # This can be 'classic_bike' or 'electric_bike'
date_to_predict = '2022-09-15'  # The date you want to make the prediction for, in 'YYYY-MM-DD' format

# Call the demand_forecasting function with the example parameters
predicted_trip_count = demand_forecasting(cluster_to_predict, bike_type_to_predict, date_to_predict)

# Print out the result
print(f"The predicted trip count for cluster {cluster_to_predict}, bike type '{bike_type_to_predict}', on {date_to_predict} is: {predicted_trip_count}")


The predicted trip count for cluster 1, bike type 'classic_bike', on 2022-09-15 is: 22.269858570955208


c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
